In [1]:
import pandas as pd

In [2]:
import numpy as np

In [4]:
data = pd.read_csv("dataset.csv", sep = ';')

In [5]:
data.head(4)

,Unnamed: 0,Adresa,Aktualizace:,Balkón:,Bankomat:,Bazén:,Bezbariérový:,Bus MHD:,Celková cena:,Cena:,...,Večerka:,Vlak:,Vlastnictví:,Voda:,Vybavení:,Výtah:,Výška stropu:,Zlevněno:,Škola:,Školka:
0,0,"Dobrovolného, Praha 9 - Černý Most Panorama",['Dnes'],NaN,"['Bankomat České spořitelny', ' (542 m)']",NaN,NaN,"['Generála Janouška', ' (132 m)']",['12 000 Kč za měsíc'],NaN,...,"['Albert Supermarket', ' (451 m)']","['Praha-Kyje', ' (1445 m)']",['Osobní'],NaN,NaN,NaN,NaN,NaN,"['ZUŠ Praha 9, Ratibořická', ' (234 m)']","['MŠ Sluníčko Praha 9 - Černý Most', ' (279 m)']"
1,0,"Bořivojova, Praha 3 - Žižkov Panorama",['Dnes'],NaN,"['Bankomat České spořitelny', ' (122 m)']",NaN,"[""'boolean-false'""]","['Tachovské náměstí', ' (443 m)']","['9 500 Kč za nemovitost, + provize RK']",NaN,...,"['Shalamar Foods', ' (165 m)']","['Praha hlavní nádraží', ' (984 m)']",['Osobní'],NaN,NaN,NaN,NaN,NaN,"['Gymnázium Karla Sladkovského Praha 3', ' (56...","['ZŠ a MŠ Jaroslava Seiferta Praha 3', ' (243 ..."
2,0,"Tupolevova, Praha 9 - Letňany Panorama",['Dnes'],NaN,"['Bankomat České spořitelny', ' (236 m)']",NaN,"[""'boolean-true'""]","['Šumperská', ' (177 m)']",['13 900 Kč za měsíc'],NaN,...,"['Penny Market', ' (183 m)']","['Praha-Čakovice', ' (1695 m)']",['Osobní'],NaN,NaN,"[""'boolean-true'""]",NaN,NaN,"['Základní škola Fryčovická', ' (160 m)']","['MŠ Havířovská', ' (255 m)']"
3,0,"Podbělohorská, Praha 5 - Smíchov Panorama",['Dnes'],NaN,"['Bankomat Komerční banky', ' (381 m)']",NaN,"[""'boolean-false'""]","['Pod Lipkami', ' (30 m)']","['15 000 Kč za nemovitost, + provize RK']",NaN,...,"['Smíšené zboží Plzeňská\t187', ' (404 m)']","['Praha-Cibulka', ' (1229 m)']",['Osobní'],['Dálkový vodovod'],NaN,NaN,NaN,NaN,"['ZUŠ Na Popelce', ' (224 m)']","['TROIS PETITES POMMES, s.r.o.', ' (343 m)']"


In [6]:
data = data.drop(["Unnamed: 0"], axis= 1)  #this drops the unnamed column as it has no value

All variables we collected

In [7]:
data.columns

Index(['Adresa', 'Aktualizace:', 'Balkón:', 'Bankomat:', 'Bazén:',
       'Bezbariérový:', 'Bus MHD:', 'Celková cena:', 'Cena:', 'Cukrárna:',
       'Datum nastěhování:', 'Datum prohlídky do:', 'Datum prohlídky:',
       'Datum zahájení prodeje:', 'Divadlo:', 'Doprava:', 'Elektřina:',
       'Energetická náročnost budovy:', 'Garáž:', 'Hospoda:', 'Hřiště:',
       'ID zakázky:', 'ID:', 'Kino:', 'Komunikace:', 'Kulturní památka:',
       'Lodžie:', 'Lékař:', 'Lékárna:', 'Metro:', 'Náklady na bydlení:',
       'Obchod:', 'Odpad:', 'Parkování:', 'Plocha podlahová:',
       'Plocha pozemku:', 'Plocha zahrady:', 'Plocha zastavěná:', 'Plyn:',
       'Podlaží:', 'Poloha domu:', 'Poznámka k ceně:', 'Počet bytů:', 'Pošta:',
       'Provize:', 'Průkaz energetické náročnosti budovy:', 'Převod do OV:',
       'Přírodní zajímavost:', 'Půdní vestavba:', 'Původní cena:',
       'Restaurace:', 'Rok kolaudace:', 'Rok rekonstrukce:', 'Sklep:',
       'Sportoviště:', 'Stav objektu:', 'Stav:', 'Stavba:', '

In [8]:
data.shape

(7878, 79)

We have 7878 observations and 80 columns

## We will examine NaN

49 variables have more than 10% of missing values

In [9]:
empty = data.isnull().sum()
empty[empty > 7878*0.1].sort_values(ascending = False).count()

49

In [10]:
empty[empty > 7878*0.1].sort_values(ascending = False)

Poloha domu:                               7877
Typ domu:                                  7877
Plocha pozemku:                            7877
Datum prohlídky do:                        7873
Datum zahájení prodeje:                    7870
Počet bytů:                                7870
Bazén:                                     7857
Převod do OV:                              7854
Půdní vestavba:                            7838
Datum prohlídky:                           7832
Cena:                                      7831
Stav:                                      7818
Plocha zahrady:                            7808
Výška stropu:                              7801
Průkaz energetické náročnosti budovy:      7719
Provize:                                   7705
Zlevněno:                                  7693
Původní cena:                              7693
Plocha zastavěná:                          7663
Rok kolaudace:                             7643
Ukazatel energetické náročnosti budovy: 

25 variables have more than 90% observations NaN. Those variables are useless

In [11]:
empty[empty > 7878*0.9].sort_values(ascending = False).count()

25

In [12]:
empty[empty > 7878*0.9].sort_values(ascending = False)

Typ domu:                                  7877
Plocha pozemku:                            7877
Poloha domu:                               7877
Datum prohlídky do:                        7873
Datum zahájení prodeje:                    7870
Počet bytů:                                7870
Bazén:                                     7857
Převod do OV:                              7854
Půdní vestavba:                            7838
Datum prohlídky:                           7832
Cena:                                      7831
Stav:                                      7818
Plocha zahrady:                            7808
Výška stropu:                              7801
Průkaz energetické náročnosti budovy:      7719
Provize:                                   7705
Původní cena:                              7693
Zlevněno:                                  7693
Plocha zastavěná:                          7663
Rok kolaudace:                             7643
Ukazatel energetické náročnosti budovy: 

We 31 variables that have sufficient data availability

In [13]:
empty[empty < 7878*0.1].sort_values(ascending = False).count()

30

In [14]:
empty[empty < 7878*0.1].sort_values(ascending = False)

Kulturní památka:    757
Kino:                463
Divadlo:             459
Obchod:              456
Restaurace:          423
Bankomat:            423
Cukrárna:            423
Hospoda:             423
Lékař:               423
Lékárna:             423
Škola:               423
Pošta:               423
Školka:              423
Veterinář:           423
Večerka:             423
Metro:               254
Celková cena:        232
Tram:                205
Vlak:                128
Bus MHD:             122
Sportoviště:         117
Hřiště:              117
Vlastnictví:           1
Podlaží:               0
Užitná plocha:         0
Title                  0
Stavba:                0
Stav objektu:          0
Aktualizace:           0
Adresa                 0
dtype: int64

The next 6 could be considered

In [15]:
empty[np.all([(empty > 7878*0.1),(empty < 7878*0.33)], axis = 0)].sort_values(ascending = False).count()

6

In [16]:
empty[np.all([(empty > 7878*0.1),(empty < 7878*0.33)], axis = 0)].sort_values(ascending = False)

Plocha podlahová:                2575
Výtah:                           2249
Poznámka k ceně:                 2114
Vybavení:                        1901
Energetická náročnost budovy:    1896
ID zakázky:                       953
dtype: int64

# Proposals for visualisation 

In [17]:
variable = empty[empty < 7878*0.33]
variable.index

Index(['Adresa', 'Aktualizace:', 'Bankomat:', 'Bus MHD:', 'Celková cena:',
       'Cukrárna:', 'Divadlo:', 'Energetická náročnost budovy:', 'Hospoda:',
       'Hřiště:', 'ID zakázky:', 'Kino:', 'Kulturní památka:', 'Lékař:',
       'Lékárna:', 'Metro:', 'Obchod:', 'Plocha podlahová:', 'Podlaží:',
       'Poznámka k ceně:', 'Pošta:', 'Restaurace:', 'Sportoviště:',
       'Stav objektu:', 'Stavba:', 'Title', 'Tram:', 'Užitná plocha:',
       'Veterinář:', 'Večerka:', 'Vlak:', 'Vlastnictví:', 'Vybavení:',
       'Výtah:', 'Škola:', 'Školka:'],
      dtype='object')

In [18]:
data[variable.index].head()

,Adresa,Aktualizace:,Bankomat:,Bus MHD:,Celková cena:,Cukrárna:,Divadlo:,Energetická náročnost budovy:,Hospoda:,Hřiště:,...,Tram:,Užitná plocha:,Veterinář:,Večerka:,Vlak:,Vlastnictví:,Vybavení:,Výtah:,Škola:,Školka:
0,"Dobrovolného, Praha 9 - Černý Most Panorama",['Dnes'],"['Bankomat České spořitelny', ' (542 m)']","['Generála Janouška', ' (132 m)']",['12 000 Kč za měsíc'],"['Shisha Room Dvin', ' (1144 m)']","['Přírodní divadlo Dády Stoklasy', ' (3152 m)']",['Třída G - Mimořádně nehospodárná'],"['Šenk Na Rajské', ' (476 m)']","['Dětské hřiště Rajský vrch', ' (162 m)']",...,"['Lehovec', ' (1136 m)']",['26m2'],"['MetropoleVet Praha s.r.o.', ' (623 m)']","['Albert Supermarket', ' (451 m)']","['Praha-Kyje', ' (1445 m)']",['Osobní'],NaN,NaN,"['ZUŠ Praha 9, Ratibořická', ' (234 m)']","['MŠ Sluníčko Praha 9 - Černý Most', ' (279 m)']"
1,"Bořivojova, Praha 3 - Žižkov Panorama",['Dnes'],"['Bankomat České spořitelny', ' (122 m)']","['Tachovské náměstí', ' (443 m)']","['9 500 Kč za nemovitost, + provize RK']","['Klub JINÝ KAFE', ' (283 m)']","['Žižkovské divadlo Járy Cimrmana', ' (280 m)']",['Třída G - Mimořádně nehospodárná'],"['Hospůdka U Habásků', ' (209 m)']","['Dětské hřiště Bořivojova', ' (294 m)']",...,"['Lipanská', ' (186 m)']",['23m2'],"['MVDr. Jan Dubský', ' (425 m)']","['Shalamar Foods', ' (165 m)']","['Praha hlavní nádraží', ' (984 m)']",['Osobní'],NaN,NaN,"['Gymnázium Karla Sladkovského Praha 3', ' (56...","['ZŠ a MŠ Jaroslava Seiferta Praha 3', ' (243 ..."
2,"Tupolevova, Praha 9 - Letňany Panorama",['Dnes'],"['Bankomat České spořitelny', ' (236 m)']","['Šumperská', ' (177 m)']",['13 900 Kč za měsíc'],"['Café Infinity', ' (204 m)']","['Divadlo Pohádka', ' (1057 m)']",NaN,"['Pivnice na Královce', ' (1824 m)']","['Venkovní posilovna u obchodního centra', ' (...",...,"['Sídliště Ďáblice', ' (1906 m)']",['52m2'],"['Veterinární ordinace Letňany', ' (302 m)']","['Penny Market', ' (183 m)']","['Praha-Čakovice', ' (1695 m)']",['Osobní'],NaN,"[""'boolean-true'""]","['Základní škola Fryčovická', ' (160 m)']","['MŠ Havířovská', ' (255 m)']"
3,"Podbělohorská, Praha 5 - Smíchov Panorama",['Dnes'],"['Bankomat Komerční banky', ' (381 m)']","['Pod Lipkami', ' (30 m)']","['15 000 Kč za nemovitost, + provize RK']","['Cukrárna Pod Marjánkou', ' (1333 m)']","['JUDr. Jiří Šilhán', ' (279 m)']",['Třída B - Velmi úsporná'],"['Restaurace Na zámyšli', ' (423 m)']","['Dětské hřiště Na Vršku', ' (116 m)']",...,"['Kavalírka', ' (370 m)']",['58m2'],"['Veterinární klinika - MVDr. Pavel Santar, MV...","['Smíšené zboží Plzeňská\t187', ' (404 m)']","['Praha-Cibulka', ' (1229 m)']",['Osobní'],NaN,NaN,"['ZUŠ Na Popelce', ' (224 m)']","['TROIS PETITES POMMES, s.r.o.', ' (343 m)']"
4,"Mezi domy, Praha 4 - Písnice Panorama",['Dnes'],"['Bankomat Komerční banky', ' (780 m)']","['U Libušské sokolovny', ' (275 m)']",['19 500 Kč za měsíc'],"['Cukrářství Viktoria, s.r.o.', ' (900 m)']","['Tradiční loutkové divadlo Zvoneček', ' (3145...",['Třída D - Méně úsporná č. 78/2013 Sb. podle ...,"['Hospůdka U Báti', ' (509 m)']","['Dětské hřiště Pyramida Na Okruhu', ' (78 m)']",...,"['Levského', ' (1815 m)']",['93m2'],"['Vetnemo, s.r.o.', ' (634 m)']","['VIVACOMEX, spol. s r.o.', ' (450 m)']","['Praha-Krč', ' (3635 m)']",['Osobní'],NaN,"[""'boolean-true'""]",['Střední odborné učiliště potravinářské Praha...,"['Mateřská škola Mezi Domy', ' (105 m)']"


In [21]:
data[variable.index].head().to_csv('dataview.csv', sep = ';')

## proposel relation for examination

celkova cena vs adresa

adresa vs metro - dokazeme urcit pozici metra pouze s techto dat pomoci kruznic a pruseciku

cena / pocet pokoju - histogram

cena / m2 - histogram

cena vs. metro - regrese, point plot 

energeticka narocnost vs adresa (jsou lokace se energetickou narocnosti shlukovane?)

### poznamky 

je vlastnictvi jine nez osobni?
vlak je podobna promenna jako metro
title ma pocet mistnosti
stavba typ - kolik je typu jak zalezi na poloze
stav objektu
poznamka k cene muze delat potize
lisi se podlahova a uzita plocha
aktualize asi neni moc zajimava promenna